In [163]:
from bs4 import BeautifulSoup
import requests
import numpy as np

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd

In [164]:
url = "https://minimalismbrand.com/"

res = requests.get(url)
print(res.status_code)
sopa = BeautifulSoup(res.content, "html.parser")


200


In [165]:
lista_links=[]
productos_hombre = sopa.findAll("ul", {"class":"navigation__tier-1"})[1]
productos_categorias= productos_hombre.findAll("li", {"class": "navigation__item navigation__item--with-children navigation__item--with-mega-menu"})
for producto in productos_categorias:
    dic_link={}
    tema = producto.find("a").text
    dic_link["tema"]=tema
    categorias = producto.findAll("li", {"class":"navigation__item navigation__item--with-children navigation__column"})
    

    diccionario_categorias={}
    # packs, prendas, accesorios
    for cate in categorias:
        nombre_cate = cate.find("a").text
        items = cate.select("ul.navigation__tier-3 li.navigation__item a.navigation__link")
        dic_items = {item.get_text(strip=True): item['href'] for item in items}

        diccionario_categorias[nombre_cate]=dic_items

        # Imprimimos los resultados
        # for categoria, enlace in categorias:
        #     print(f"Categoría: {categoria}, Enlace: {enlace}")
    
    dic_link["categorias"]=diccionario_categorias
    lista_links.append(dic_link)
                
lista_links_final = lista_links[:2] 
#print(productos_hombre_categorias[0])


In [166]:
#lista_links_final

In [168]:
url_base = "https://minimalismbrand.com/"

for elem in lista_links_final:
    for categoria, valor in elem["categorias"].items():
        print(categoria)
        if categoria == "Packs más vendidos":
                print("fuera")
                continue

        for item, link in valor.items():
            if item == "Tarjeta regalo" or item == "Fundas de portátil" or item=="Fundas portátil":
                print("NO")

            else:    
                print(f"   {item}")
                print(f"   {link}\n")

                url = url_base+link
                res = requests.get(url)
                sopa = BeautifulSoup(res.content, "html.parser")

                prendas = sopa.find("div", {"class":"product-list product-list--per-row-3 product-list--per-row-mob-2 product-list--per-row-mob-2 product-list--image-shape-shortest"})
                links_productos = []

                # Buscar todas las etiquetas <a> con la clase "product-link"
                productos = prendas.findAll("a", {"class": 'product-link'})
                
                enlaces_productos = [producto['href'] for producto in productos]

                elem["categorias"][categoria][item]=enlaces_productos
        



Packs más vendidos
fuera
Prendas
   Camisetas y polos
   /collections/camisetas-algodon-organico

   Calzoncillos
   /collections/calzoncillos

   Calcetines
   /collections/calcetines-algodon-organico

   Sudaderas
   /collections/sudaderas

   Jerséis
   /collections/sweater-men

   Pantalones
   /collections/pantalones-organicos-hombre

   Bañadores
   /collections/banadores

NO
Accesorios
   Mochilas
   /collections/mochilas

NO
   Carteras
   /collections/carteras

   Gorro
   /collections/gorro

   Bufanda
   /collections/bufanda

   Guantes
   /collections/guantes

Packs más vendidos
fuera
Prendas
   Camisetas
   /collections/camisetas-algodon-organico-mujer

   Ropa interior
   /collections/pack-ropa-interior-mujer-minimalista

   Calcetines
   /collections/calcetines-algodon-organico

   Sudaderas
   /collections/sudaderas-mujer

   Jerséis
   /collections/sweater-women

   Pantalones
   /collections/pantalones-organicos-mujer

NO
Accesorios
   Mochilas
   /collections/mochila

In [173]:
import json
with open('diccionario.json', 'w') as archivo:
    json.dump(lista_links_final, archivo, indent=4)

In [174]:
with open('diccionario.json', 'r') as archivo:
    lista_links_final = json.load(archivo)

In [241]:
url_cami = "https://minimalismbrand.com/products/pantalon-organico-semi-skinny-hombre"
res = requests.get(url_cami)
sopa = BeautifulSoup(res.content, "html.parser")

# Nombre producto
titulo = sopa.find("div", {"class":"title-row"}).text
print(titulo)

precio = sopa.find("div", {"class":"price-area"}).text.split()[0]
print(precio)

if "pack" in titulo.lower():
    nombres_colores=[]
    nombres_tallas=[]

else:
    if len(sopa.findAll("div", {"class":"option-selector__btns"}))==1:

        div_tallas = sopa.findAll("div", {"class":"option-selector__btns"})[0]
        nombres_tallas = [label.get_text() for label in div_tallas.find_all("span", class_="opt-label__text")]
        print(nombres_tallas)

        nombres_colores = []
        

    else:
        # Colores y tallas
        div_colores = sopa.findAll("div", {"class":"option-selector__btns"})[0]
        div_tallas = sopa.findAll("div", {"class":"option-selector__btns"})[1]

        nombres_colores = [label.get_text() for label in div_colores.find_all("span", class_="opt-label__text")]
        print(nombres_colores)
        nombres_tallas = [label.get_text() for label in div_tallas.find_all("span", class_="opt-label__text")]
        print(nombres_tallas)

# Descripción
parr = sopa.find("div", {"class":"cc-accordion-item__content rte cf"})
descripcion = parr.text

# Impacto 
impacto = sopa.find("span", {"class":"metafield-multi_line_text_field"}).text
print(impacto)




Pantalón orgánico Slim Hombre

€70.00
['38', '40', '42', '44', '46']
10,7 kg de emisiones de CO2 evitados
2.985 litros de agua ahorrados
35,4 kWh de energía ahorrados


In [248]:
diccionario_csv = {
    "tema": [],
    "categoria": [],
    "subcategoria": [],
    "nombre":[],
    "precio":[],
    "colores":[],
    "tallas":[],
    "descripcion":[],
    "imapcto":[]
}

for elem in lista_links_final:
    #diccionario_csv["tema"].append(elem["tema"])

    # categoria: prendas, packs, accesorios
    for categoria, valor in elem["categorias"].items():

        # Se gestiona a parte
        if categoria == "Packs más vendidos":
                print("fuera")
                continue
        
        # diccionario_csv["categoria"]=categoria

        #---------------ITEMS-------------------
        for item, lista_links in valor.items():

            if item == "Tarjeta regalo" or item == "Fundas de portátil" or item=="Fundas portátil":
                print("NO")
            
            else:
                #diccionario_csv["subcategorias"]=item

                for link in lista_links:
                    print(link)
                    url_item = "https://minimalismbrand.com"+link
                    res = requests.get(url_item)
                    sopa = BeautifulSoup(res.content, "html.parser")

                    # Nombre producto
                    titulo = sopa.find("div", {"class":"title-row"}).text
                    #print(titulo)

                    precio = sopa.find("div", {"class":"price-area"}).text.split()[0]
                    #print(precio)

                    terminos_buscar = ["pack", "x2", "x3", "x5", "x7", "mochila", "bolso", "cartera", "gorro"]
                    if any(termino in titulo.lower() for termino in terminos_buscar):
                        nombres_colores=[]
                        nombres_tallas=[]

                    else:

                        if len(sopa.findAll("div", {"class":"option-selector__btns"}))==1:

                            div_tallas = sopa.findAll("div", {"class":"option-selector__btns"})[0]
                            nombres_tallas = [label.get_text() for label in div_tallas.find_all("span", class_="opt-label__text")]
                            print(nombres_tallas)

                            nombres_colores = []

                        else:
                            # Colores y tallas
                            div_colores = sopa.findAll("div", {"class":"option-selector__btns"})[0]
                            div_tallas = sopa.findAll("div", {"class":"option-selector__btns"})[1]

                            nombres_colores = [label.get_text() for label in div_colores.find_all("span", class_="opt-label__text")]
                            #print(nombres_colores)
                            nombres_tallas = [label.get_text() for label in div_tallas.find_all("span", class_="opt-label__text")]
                            #print(nombres_tallas)

                    # Descripción
                    parr = sopa.find("div", {"class":"cc-accordion-item__content rte cf"})
                    descripcion = parr.text

                    # Impacto
                    try:
                        impacto = sopa.find("span", {"class":"metafield-multi_line_text_field"}).text
                        #print(impacto)
                    except:
                        impacto=""

                    diccionario_csv["tema"].append(elem["tema"])
                    diccionario_csv["categoria"].append(categoria)
                    diccionario_csv["subcategoria"].append(item)
                    diccionario_csv["nombre"].append(titulo)
                    diccionario_csv["precio"].append(precio)
                    diccionario_csv["colores"].append(nombres_colores)
                    diccionario_csv["tallas"].append(nombres_tallas)
                    diccionario_csv["descripcion"].append(descripcion)
                    diccionario_csv["imapcto"].append(impacto)



fuera
/products/camiseta-algodon-organico?variant=34948534108317
/products/camiseta-algodon-organico?variant=34948534108317
/products/camiseta-algodon-organico?variant=34948524540061
/products/camiseta-algodon-organico?variant=34948524540061
/products/camiseta-algodon-organico?variant=47057331650895
/products/camiseta-algodon-organico?variant=47057331650895
/products/camiseta-algodon-organico?variant=34948530765981
/products/camiseta-algodon-organico?variant=34948530765981
/products/camiseta-algodon-organico?variant=48043523866959
/products/camiseta-algodon-organico?variant=48043523866959
/products/camiseta-algodon-organico?variant=34948537417885
/products/camiseta-algodon-organico?variant=34948537417885
/products/camiseta-algodon-organico?variant=34948539482269
/products/camiseta-algodon-organico?variant=34948539482269
/products/camiseta-algodon-organico?variant=48043509055823
/products/camiseta-algodon-organico?variant=48043509055823
/products/camiseta-algodon-organico?variant=490480

In [249]:
df_hombre_mujer = pd.DataFrame(diccionario_csv)
df_hombre_mujer.to_csv("../datos/df_hombre_mujer.csv")

In [250]:
df_hombre_mujer.columns

Index(['tema', 'categoria', 'subcategoria', 'nombre', 'precio', 'colores',
       'tallas', 'descripcion', 'imapcto'],
      dtype='object')

In [259]:
df_hombre_mujer["imapcto"].unique()

array([' \n¿Por qué decimos que esta camiseta es sostenible?\n\n- Es confeccionada con algodón 100% orgánico\n- Está fabricada en el norte de Portugal, a pocos kilómetros de nuestro almacén y de nuestro consumidor final\n- Al emplear materiales de primera calidad su durabilidad incrementa enormemente\n- Empleamos packaging totalmente reciclado y reciclable\n\n\nElegir la camiseta Minimalism, fabricada con algodón orgánico certificado (OCS 100), en lugar de una camiseta genérica supone ahorrar:\n\n- 2,4 kg de emisiones de CO2\n- 614,9 litros de agua\n- 3,8 kWh de energía\n\nDatos de sostenibilidad auditados por Green Story',
       ' \n¿Por qué decimos que nuestras camisetas son sostenibles?\n\n- Están confeccionadas con algodón 100% orgánico\n- Se fabrican en el norte de Portugal, a pocos kilómetros de nuestro almacén y de nuestro consumidor final\n- Al emplear materiales de primera calidad su durabilidad incrementa enormemente\n- Empleamos packaging totalmente reciclado y reciclable\n

In [274]:
df_hombre_mujer.head(1)

,tema,categoria,subcategoria,nombre,precio,colores,tallas,descripcion,imapcto
0,Hombre,Prendas,Camisetas y polos,\nCamiseta algodón orgánico negra\n,€25.00,"[Negra, Blanca, Crudo, Gris Jaspeado, Lava Gre...","[XS, S, M, L, XL, XXL]",Camisetas de algodón orgánico negra. Al emplea...,\n¿Por qué decimos que esta camiseta es soste...


In [269]:
df_hombre_mujer.duplicated(["tema", "nombre", "precio", "descripcion"]).sum()

np.int64(313)

In [272]:
df_sin_duplicados = df_hombre_mujer.drop_duplicates(["tema","categoria", "subcategoria", "nombre"])

In [275]:
df_sin_duplicados.to_csv("../datos/df_sin_duplicados.csv")